# Session 5

First finished Session 4 - Looking at logical condistions

In [ ]:
import pandas as pd

filepath = "https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"

dfs = pd.read_excel(filepath, sheet_name=None) #read all sheets


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CHILD   2676 non-null   object 
 1   SEX     2676 non-null   int64  
 2   DOB     2676 non-null   object 
 3   ETHNIC  2676 non-null   object 
 4   UPN     2676 non-null   object 
 5   MOTHER  1 non-null      float64
 6   MC_DOB  1 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 146.5+ KB


In [12]:
dfs.keys()

# dfs['header'].info()

# dfs['header'].head()

dict_keys(['ad1', 'episodes', 'header', 'missing', 'oc2', 'oc3', 'placed_for_adoption', 'previous_permenance', 'reviews', 'uasc'])

In [ ]:
# uses DD/MM/YYYY
#taking header out of dict
header = dfs['header']

#ssda903  102 rule / erros  says that date of birth is not a valid date

header['DOB_dt'] =pd.to_datetime(header['DOB'], format =r"%d/%m/%Y", errors = 'coerce')
#coercing errors returns NaTs for invalid dates 
#so if i slice out nat i have all invalid dates. 

invalid_dobs = header['DOB_dt'].isna() # returns true when value in column is an NA

error_rows = header[invalid_dobs]

error_rows # this is wjy we add columns instead of replacing them. so we can check errrors and refer back



,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


## Task making own data validation rules

In [33]:
## 115 - Date of Local Authorities (LA) decision that a child should be placed for adoption is not a valid date

#select placed for adoption tab;le from dfs,
# # convert the correct column to a datetime, 
# make the slice

placed_for_adoption = dfs['placed_for_adoption']

#placed_for_adoption.info()
#placed_for_adoption.head()

placed_for_adoption['DATE_PLACED_dt'] = pd.to_datetime(placed_for_adoption['DATE_PLACED'], format =r"%d/%m/%Y", errors = 'coerce')

invalid_placed_for_adoption = placed_for_adoption['DATE_PLACED_dt'].isna()

error_rows_adoption = placed_for_adoption[invalid_placed_for_adoption]
# error_rows_adoption = placed_for_adoption[placed_for_adoption['DATE_PLACED_df'].isna()]

error_rows_adoption.head() # this returns 1 row which can be removed as it is a fake entry

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_df,DATE_PLACED_dt
29,bad_entry_1,Not a date,This isn't a valid date,This also isn't a date,oop,NaT,NaT


In [ ]:
#break until 5 past

# .isin() -> returns true where a column has a value in a list... it is in the list
# 103 - The ethnicity code is either not valid or has not been entered.

ethnicity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
]

error_rows = header[~ header['ETHNIC'].isin(ethnicity_codes)]  # Using ~ to say not inthe list. 
error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB,DOB_dt
2675,bad_entry_1,7,oops!,Alien,UPDOG,NaN,NaN,NaT


In [ ]:
#excerise
# 114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
# When we do an isin we need to check the datatype of the list is the same as the values
# valid codes 0, 1
# table is ad1

ad1 = dfs['ad1']

ad1.info() # check datatype of column
ad1.head() # looking at FOSTER_CARE


error_rows_ad1 = ad1[~ ad1['FOSTER_CARE'].isin([0,1])] # as there are only 2 valid entries I just wrote the list in this line
error_rows_ad1

## if FOSTER_CARE Wasn#t a int64 object would need to cast it

ad1['FOSTER_CARE'] = ad1['FOSTER_CARE'].astype('str')

error_rows_ad1 = ad1[~ ad1['FOSTER_CARE'].isin(["0","1"])]
error_rows_ad1


#can sort columns numerically and alphabetically. 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CHILD        30 non-null     object
 1   DOB          30 non-null     object
 2   DATE_INT     30 non-null     object
 3   DATE_MATCH   30 non-null     object
 4   FOSTER_CARE  30 non-null     object
 5   NB_ADOPTR    30 non-null     object
 6   SEX_ADOPTR   30 non-null     object
 7   LS_ADOPTR    30 non-null     object
dtypes: object(8)
memory usage: 2.0+ KB


,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [37]:
# 184 date of decision that a child should be placed for adoption is before child was born


# need placed for adoption 
# we don't need header as dob is already on their  but going to use it for the example

merged_df = placed_for_adoption.merge(header[['CHILD', 'DOB_dt']], on='CHILD', how= 'left', suffixes = [None, "_head"]) 
#Can use double square brackets to only just a subet of columns froim the right table
#First sqyare bracket is saying fro  header. second square bracket is saying from the header give me this list. 


merged_df.info()
merged_df.head()

error_rows_merge = merged_df[merged_df['DOB_dt']> merged_df['DATE_PLACED_dt']] 

error_rows_merge

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CHILD                 30 non-null     object        
 1   DOB                   30 non-null     object        
 2   DATE_PLACED           30 non-null     object        
 3   DATE_PLACED_CEASED    30 non-null     object        
 4   REASON_PLACED_CEASED  30 non-null     object        
 5   DATE_PLACED_df        29 non-null     datetime64[ns]
 6   DATE_PLACED_dt        29 non-null     datetime64[ns]
 7   DOB_dt                29 non-null     datetime64[ns]
dtypes: datetime64[ns](3), object(5)
memory usage: 2.0+ KB


,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,DATE_PLACED_df,DATE_PLACED_dt,DOB_dt


In [ ]:
# Excercise 
# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes

dfs.keys()

#need header of Dobs
#need episodes for episodes.,

episodes = dfs['episodes']
#episodes.info()
#episodes.head()

episodes['DECOM_dt'] = pd.to_datetime(episodes['DECOM'], format = r"%d/%m/%Y", errors='coerce')

error_rows = episodes[episodes['DECOM_dt'].isna()]

error_rows # no errors. 

eps_head_df = episodes.merge(header[['CHILD', 'DOB_dt']], on='CHILD', how = 'left', suffixes= [None, "_head"])


invalid_302 = eps_head_df['DECOM_dt'] < eps_head_df['DOB_dt']

errors_103 = eps_head_df[invalid_302]

errors_103


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CHILD                5050 non-null   int64         
 1   DECOM                5050 non-null   object        
 2   RNE                  5050 non-null   object        
 3   LS                   5050 non-null   object        
 4   CIN                  5050 non-null   object        
 5   PLACE                5050 non-null   object        
 6   PLACE_PROVIDER       5050 non-null   object        
 7   DEC                  3637 non-null   object        
 8   REC                  5050 non-null   object        
 9   REASON_PLACE_CHANGE  2817 non-null   object        
 10  HOME_POST            5049 non-null   object        
 11  PL_POST              5049 non-null   object        
 12  URN                  5050 non-null   int64         
 13  DECOM_dt             5050 non-nul

,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,DECOM_dt,DOB_dt


In [54]:
#sort_values()

epi_ordered = episodes.sort_values(['CHILD', 'DECOM_dt'], ascending = True) 
epi_ordered # This is now orderd by child then by date

#we only want to keep the first row of every child. 

first_episode = epi_ordered.drop_duplicates('CHILD', keep = 'first')

first_episode

,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,DECOM_dt
4801,32,12/02/2017,T,J1,N3,K2,PR1,15/04/2017,E3,NaN,ED8 1OX,WI5 9CC,7876040,2017-02-12
4041,982,27/11/2016,P,E1,N7,A5,PR3,07/10/2017,X1,LAREQ,EP10 1XZ,WI5 8XU,6162294,2016-11-27
437,1243,02/01/2017,L,V4,N5,Z1,PR0,18/05/2017,E9,NaN,EC5 5MZ,WC30 9OS,7270736,2017-01-02
1794,1407,28/11/2015,L,L2,N3,P2,PR1,29/04/2017,X1,ALLEG,EG16 2ZL,WU19 1JS,6492318,2015-11-28
3848,2431,20/02/2016,S,V2,N3,P2,PR5,18/11/2017,X1,APPRR,EK12 3JI,WV1 1FJ,5227516,2016-02-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,997201,18/05/2016,P,L3,N4,T3,PR3,07/06/2017,X1,CUSTOD,EL12 3BA,WV22 6GC,3780837,2016-05-18
2766,997445,20/06/2016,P,J1,N2,U3,PR3,24/09/2017,E48,NaN,EP2 2VO,WE6 1FZ,3786993,2016-06-20
154,998192,27/11/2015,P,C2,N2,P1,PR5,08/07/2017,X1,ALLEG,EJ11 4FS,WT21 5RE,6722585,2015-11-27
156,998777,11/10/2015,P,C2,N2,U1,PR2,17/03/2018,X1,CLOSE,ER24 5VS,WJ7 5PQ,1112628,2015-10-11
